In [ ]:
!pip install spacy
!pip install nltk

In [ ]:
!python -m spacy download en_core_web_lg

2024-01-28 19:05:22.355814: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 19:05:22.355874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 19:05:22.357373: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 19:05:22.366477: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 19:05:24.069769: W tensorflow/compiler/tf2

In [ ]:
!pip install -q vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn import metrics
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load the CSV file into a DataFrame
rra = r'/content/drive/MyDrive/restaurant_reviews_az.csv'
df= pd.read_csv(rra)

# Summary
print(df.head(5))
print(df.shape)
print(df.describe())

                review_id                 user_id             business_id  \
0  IVS7do_HBzroiCiymNdxDg  fdFgZQQYQJeEAshH4lxSfQ  sGy67CpJctjeCWClWqonjA   
1  QP2pSzSqpJTMWOCuUuyXkQ  JBLWSXBTKFvJYYiM-FnCOQ  3w7NRntdQ9h0KwDsksIt5Q   
2  oK0cGYStgDOusZKz9B1qug  2_9fKnXChUjC5xArfF8BLg  OMnPtRGmbY8qH_wIILfYKA   
3  E_ABvFCNVLbfOgRg3Pv1KQ  9MExTQ76GSKhxSWnTS901g  V9XlikTxq0My4gE8LULsjw   
4  Rd222CrrnXkXukR2iWj69g  LPxuausjvDN88uPr-Q4cQA  CA5BOxKRDPGJgdUQ8OUOpw   

   stars  useful  funny  cool  \
0      3       1      1     0   
1      5       1      1     1   
2      5       1      0     0   
3      5       0      0     0   
4      4       1      0     0   

                                                text                 date  
0  OK, the hype about having Hatch chili in your ...  2020-01-27 22:59:06  
1  Pandemic pit stop to have an ice cream.... onl...  2020-04-19 05:33:16  
2  I was lucky enough to go to the soft opening a...  2020-02-29 19:43:44  
3  I've gone to claim Jumpers all 

In [ ]:
# Remove 3-star reviews
df = df[df['stars'] != 3]

# Create a new column 'Sentiment' based on the rating
df['Sentiment'] = df['stars'].apply(lambda x: 0 if x in [1, 2] else 1)


print(df.head())

                review_id                 user_id             business_id  \
1  QP2pSzSqpJTMWOCuUuyXkQ  JBLWSXBTKFvJYYiM-FnCOQ  3w7NRntdQ9h0KwDsksIt5Q   
2  oK0cGYStgDOusZKz9B1qug  2_9fKnXChUjC5xArfF8BLg  OMnPtRGmbY8qH_wIILfYKA   
3  E_ABvFCNVLbfOgRg3Pv1KQ  9MExTQ76GSKhxSWnTS901g  V9XlikTxq0My4gE8LULsjw   
4  Rd222CrrnXkXukR2iWj69g  LPxuausjvDN88uPr-Q4cQA  CA5BOxKRDPGJgdUQ8OUOpw   
5  kx6O_lyLzUnA7Xip5wh2NA  YsINprB2G1DM8qG1hbrPUg  rViAhfKLKmwbhTKROM9m0w   

   stars  useful  funny  cool  \
1      5       1      1     1   
2      5       1      0     0   
3      5       0      0     0   
4      4       1      0     0   
5      1       0      0     0   

                                                text                 date  \
1  Pandemic pit stop to have an ice cream.... onl...  2020-04-19 05:33:16   
2  I was lucky enough to go to the soft opening a...  2020-02-29 19:43:44   
3  I've gone to claim Jumpers all over the US and...  2020-03-14 21:47:07   
4  If you haven't been  to May

In [ ]:
# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, Y_train, Y_test = train_test_split(
    df['text'], df['Sentiment'], test_size=0.2, random_state=42)

# Display the size of training and testing sets
print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))

Training set size: 35274
Testing set size: 8819


In [ ]:
# Use Count Vectorizer with a maximum feature size of 1000
count_vectorizer = CountVectorizer(max_features=1000)
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)


In [ ]:
# Train Naive Bayes Classification Model
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_count, Y_train)
nb_predictions = nb_classifier.predict(X_test_count)
nb_accuracy = metrics.accuracy_score(Y_test, nb_predictions)
print("Naive Bayes Accuracy:", nb_accuracy)


Naive Bayes Accuracy: 0.9209660959292437


In [ ]:
# Train SVM Model
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_count, Y_train)
svm_predictions = svm_classifier.predict(X_test_count)
svm_accuracy = metrics.accuracy_score(Y_test, svm_predictions)
print("SVM Accuracy:", svm_accuracy)

SVM Accuracy: 0.9468193672752012


In [ ]:
# Use TF-IDF Vectorizer with a maximum feature size of 1000
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# Training a naive bayes classifcation model with TF-IDF feature values
nb_tfidf_classifier = MultinomialNB()
nb_tfidf_classifier.fit(X_train_tfidf, Y_train)
nb_tfidf_predictions = nb_tfidf_classifier.predict(X_test_tfidf)
nb_tfidf_accuracy = metrics.accuracy_score(Y_test, nb_tfidf_predictions)
print("Naive Bayes with TF-IDF Accuracy:", nb_tfidf_accuracy)

Naive Bayes with TF-IDF Accuracy: 0.9070189363873455


In [ ]:
# Train SVM Model with TF-IDF
svm_tfidf_classifier = SVC(kernel='linear')
svm_tfidf_classifier.fit(X_train_tfidf, Y_train)
svm_tfidf_predictions = svm_tfidf_classifier.predict(X_test_tfidf)
svm_tfidf_accuracy = metrics.accuracy_score(Y_test, svm_tfidf_predictions)
print("SVM with TF-IDF Accuracy:", svm_tfidf_accuracy)

SVM with TF-IDF Accuracy: 0.9492005896360132


In [ ]:
# Use VaderSentiment analyzer
analyzer = SentimentIntensityAnalyzer()
vader_predictions = X_test.apply(lambda x: 1 if analyzer.polarity_scores(x)['compound'] >= 0 else 0)
vader_accuracy = metrics.accuracy_score(Y_test, vader_predictions)
print("VaderSentiment Accuracy:", vader_accuracy)

VaderSentiment Accuracy: 0.862456060777866


Naive Bayes with Count Vectorizer:

Accuracy: 0.92

SVM with Count Vectorizer:

Accuracy: 0.95 (0.946)

Naive Bayes with TF-IDF:

Accuracy: 0.91

SVM with TF-IDF:

Accuracy: 0.95(0.949)

VaderSentiment:

Accuracy: 0.86

My observation is that Machine learning models, especially SVM with TF-IDF, outperform lexicon-based approach. SVM models, whether with Count Vectorizer or TF-IDF, consistently show high accuracy, indicating their effectiveness. Naive Bayes models perform well but may slightly lag behind SVM in capturing complex sentiments. VaderSentiment, being lexicon-based, resulting in lower accuracy.
In conclusion, for better accuracy and nuanced sentiment analysis, machine learning models, particularly SVM with TF-IDF, are preferred over lexicon-based approaches like VaderSentiment.







In [ ]:
!pip install -q jupyter
!pip install -q nbconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.5 MB/s eta 0:00:00
